In [117]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
pd.set_option('display.max_columns', None)
warnings.filterwarnings("ignore")
from tslearn.clustering import TimeSeriesKMeans, KShape
from tslearn.datasets import CachedDatasets
from tslearn.preprocessing import TimeSeriesScalerMeanVariance, \
    TimeSeriesResampler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet, SGDRegressor
from sklearn.svm import SVR, LinearSVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from catboost import CatBoostRegressor
from sklearn.cluster import KMeans
from kneed import DataGenerator, KneeLocator
from datetime import timedelta, datetime

In [118]:
GenerationBG = pd.read_csv("BG_20140101_20220101_Load_Generation.csv")

In [119]:
DR = pd.date_range(start=GenerationBG.Date.min(),end=GenerationBG.Date.max(),freq="60T")

In [120]:
GenerationBG = pd.read_csv("BG_20140101_20220101_Load_Generation.csv")
GenerationBG.drop(0,inplace=True)
GenerationBG.drop("Unnamed: 0",axis=1,inplace=True)
GenerationBG['Date'] = pd.to_datetime(GenerationBG['Date'],utc=True)
GenerationBG[[i for i in GenerationBG.columns if "Date" not in i]] = GenerationBG[[i for i in GenerationBG.columns if "Date" not in i]].astype(float)
GenerationBG1 = GenerationBG[["Date","Solar","Wind Onshore"]]
GenerationBG1["Renewable Total"] = GenerationBG1["Solar"] + GenerationBG1["Wind Onshore"]
GenerationBG1 = GenerationBG1.set_index("Date").reindex(DR)

In [121]:
Load_BG = pd.read_feather("BG_20140101_20220101_Load.feather")
Load_BG1=Load_BG.set_index("Date").reindex(DR)

In [122]:
Day_ahead_price_BG = pd.read_feather("BG_20140101_20220101_DayAheadPrices.feather")
Day_ahead_price_BG1 = Day_ahead_price_BG.set_index("Date").reindex(DR)

In [123]:
Load_Forecast_BG = pd.read_feather("BG_20140101_20220101_Load_Forecast.feather")
Load_Forecast_BG1 = Load_Forecast_BG.set_index("Date").reindex(DR)

In [124]:
Net_position_BG = pd.read_feather("BG_20140101_20220101_Net_Position.feather")
Net_position_BG1 = Net_position_BG.set_index("Date").reindex(DR)

In [125]:
SW_forecast_BG = pd.read_feather("BG_20140101_20220101_SW_Forecast.feather")
SW_forecast_BG1 = SW_forecast_BG.set_index("Date").reindex(DR)
SW_forecast_BG1 = SW_forecast_BG1.add_suffix("_FC")

In [126]:
Total_BG = pd.concat([GenerationBG1,Load_BG1,Day_ahead_price_BG1,Load_Forecast_BG1,Net_position_BG1,SW_forecast_BG1],axis=1)

In [127]:
# Total_BG = Total_BG.interpolate(method='linear', limit_direction='forward', axis=1)
# Total_BG = Total_BG.fillna(0)

In [128]:
Total_BG.reset_index(inplace=True)

In [129]:
Total_BG.rename({"index":"Date"},axis=1,inplace=True)

In [130]:
Total_BG

,Date,Solar,Wind Onshore,Renewable Total,Load_,Prices_BG,Forecasted Load,Net_Position_BG,Solar_FC,Wind Onshore_FC
0,2020-01-01 01:00:00+02:00,0.0,0.0,0.0,4243.0,76.26,4444.0,2222.0,0.0,362.0
1,2020-01-01 02:00:00+02:00,0.0,323.0,323.0,4063.0,68.57,4234.0,2117.0,0.0,318.0
2,2020-01-01 03:00:00+02:00,0.0,306.0,306.0,3924.0,58.99,4064.0,2032.0,0.0,370.0
3,2020-01-01 04:00:00+02:00,0.0,351.0,351.0,3838.0,52.51,3942.0,1971.0,0.0,404.0
4,2020-01-01 05:00:00+02:00,0.0,431.0,431.0,3795.0,57.23,3882.0,1941.0,0.0,423.0
...,...,...,...,...,...,...,...,...,...,...
17539,2021-12-31 20:00:00+02:00,0.0,137.0,137.0,4790.0,221.01,4839.0,744.0,0.0,204.0
17540,2021-12-31 21:00:00+02:00,0.0,145.0,145.0,4343.0,205.37,4602.0,756.0,0.0,211.0
17541,2021-12-31 22:00:00+02:00,0.0,150.0,150.0,4196.0,194.35,4558.0,700.0,0.0,204.0
17542,2021-12-31 23:00:00+02:00,0.0,136.0,136.0,4147.0,205.38,4355.0,737.0,0.0,157.0


In [131]:
Total_BG.to_feather("Total_BG.feather")